In [4]:
import os
import cv2
import shutil
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset, DataLoader
from PIL import Image


## APPLYING SAME AUGMENTATION ON THE DATA AS WAS USED TO TRAIN THE MODEL ##

In [5]:
#-----------------------------------------------------------------------------------
#  Data Organistation
def create_directory_structure(base_path):
    """Create the necessary directory structure for the dataset."""

    folders = [
        "train/images", "train/labels",
        "val/images", "val/labels",
        "test/images", "test/labels",
        "unlabeled/images"
    ]

    for folder in folders:
        os.makedirs(os.path.join(base_path, folder), exist_ok=True)
        
    return folders
#------------------------------------------------------------------------------------

# Data augmentation
def adjust_annotations(annotations, crop_x, crop_y, crop_w, crop_h, orig_w, orig_h):
    """Adjust bounding box annotations after cropping."""
    new_annotations = []
    for line in annotations:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        x_abs, y_abs = x_center * orig_w, y_center * orig_h
        width, height = width * orig_w, height * orig_h
        
        x1, y1 = x_abs - width / 2, y_abs - height / 2
        x2, y2 = x_abs + width / 2, y_abs + height / 2
        
        if x1 >= crop_x and y1 >= crop_y and x2 <= crop_x + crop_w and y2 <= crop_y + crop_h:
            x1_new, y1_new = x1 - crop_x, y1 - crop_y
            x2_new, y2_new = x2 - crop_x, y2 - crop_y
            x_center_new = (x1_new + x2_new) / 2 / crop_w
            y_center_new = (y1_new + y2_new) / 2 / crop_h
            width_new = (x2_new - x1_new) / crop_w
            height_new = (y2_new - y1_new) / crop_h
            new_annotations.append(f"{class_id} {x_center_new:.6f} {y_center_new:.6f} {width_new:.6f} {height_new:.6f}")
    return new_annotations

def random_crop(image, annotations, crop_size=(224, 224)):
    """Randomly crop image and adjust annotations."""
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        return image, annotations
    x_start = random.randint(0, w - crop_w)
    y_start = random.randint(0, h - crop_h)
    cropped_image = image[y_start:y_start + crop_h, x_start:x_start + crop_w]
    new_annotations = adjust_annotations(annotations, x_start, y_start, crop_w, crop_h, w, h)
    return cropped_image, new_annotations

def invert_colors(image):
    """Invert image colors."""
    return cv2.bitwise_not(image)

def apply_gaussian_blur(image, kernel_size=(5, 5)):
    """Apply Gaussian blur to image."""
    return cv2.GaussianBlur(image, kernel_size, 0)

    
def process_training_set(train_orig, source_paths, dest_paths, augment=True):
    """Process training set with optional augmentation."""
    processed_count = 0
    augmented_count = 0
    
    for img_name in tqdm(train_orig, desc="Processing training images"):
        img_path = os.path.join(source_paths['images'], img_name)
        ann_path = os.path.join(source_paths['annotations'], img_name.replace(".jpg", ".txt"))
        
        if not os.path.exists(ann_path):
            continue
        
        image = cv2.imread(img_path)
        with open(ann_path, "r") as f:
            annotations = f.readlines()
        
        
        shutil.copy(img_path, os.path.join(dest_paths['images'], img_name))
        shutil.copy(ann_path, os.path.join(dest_paths['labels'], img_name.replace(".jpg", ".txt")))
        processed_count += 1
        
        if augment:
            augmentations = [
                random_crop(image, annotations),
                (invert_colors(image), annotations),
                (apply_gaussian_blur(image), annotations)
            ]
            
            for i, (aug_img, aug_ann) in enumerate(augmentations):
                aug_img_name = f"aug_{i}_{img_name}"
                aug_ann_name = f"aug_{i}_{img_name.replace('.jpg', '.txt')}"
                cv2.imwrite(os.path.join(dest_paths['images'], aug_img_name), aug_img)
                with open(os.path.join(dest_paths['labels'], aug_ann_name), "w") as f:
                    f.write("\n".join(aug_ann))
                augmented_count += 1
                
    return processed_count, augmented_count

def process_unlabeled_set(unlabeled_images, source_paths, dest_paths):
    """Process unlabeled images (without annotations)."""
    processed_count = 0
    
    for img_name in tqdm(unlabeled_images, desc="Processing unlabeled images"):
        img_path = os.path.join(source_paths['unlabeled_images'], img_name) 
        if not os.path.exists(img_path):
            continue
        
        
        shutil.copy(img_path, os.path.join(dest_paths['images'], img_name))
        processed_count += 1
    
    return processed_count
#-----------------------------------------------------------------------------------

# Data Splitting and Statistics 
def process_dataset_split(image_list, source_paths, dest_paths, split_name=""):
    """Process dataset split (validation or test)."""
    processed_count = 0
    for img_name in tqdm(image_list, desc=f"Processing {split_name} images"):
        img_path = os.path.join(source_paths['images'], img_name)
        ann_path = os.path.join(source_paths['annotations'], img_name.replace(".jpg", ".txt"))
        
        if not os.path.exists(ann_path):
            continue
        
        shutil.copy(img_path, os.path.join(dest_paths['images'], img_name))
        shutil.copy(ann_path, os.path.join(dest_paths['labels'], img_name.replace(".jpg", ".txt")))
        processed_count += 1
    
    return processed_count

def get_dir_size(path):
    """Get directory size in MB."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)
#------------------------------------------------------------------------------------    

def main():
    # Define paths
    base_path = "/kaggle/input/weedzip"
    source_paths = {
        'labeled_images': os.path.join(base_path, "labeled/images"),
        'labeled_annotations': os.path.join(base_path, "labeled/annotations"),
        'test_images': os.path.join(base_path, "test/images"),
        'test_annotations': os.path.join(base_path, "test/annotations"),
        'unlabeled_images': os.path.join(base_path, "unlabeled")
    }
    
    preprocessed_path = "/kaggle/working/Preprocessed_img_weed"
    folders = create_directory_structure(preprocessed_path)
    
    # Split original dataset
    all_original_images = os.listdir(source_paths['labeled_images'])
    train_orig, val_orig = train_test_split(all_original_images, test_size=0.2, random_state=42)
    
    # Split unlabeled dataset
    unlabeled_images = os.listdir(source_paths['unlabeled_images'])
    
    # Print initial statistics
    print("\nOriginal dataset statistics:")
    print(f"Total original images: {len(all_original_images)}")
    print(f"Training images before augmentation: {len(train_orig)}")
    print(f"Validation images: {len(val_orig)}")
    print(f"Unlabeled images: {len(unlabeled_images)}")
    
    # Process training set
    train_paths = {
        'images': os.path.join(preprocessed_path, "train/images"),
        'labels': os.path.join(preprocessed_path, "train/labels")
    }
    processed_train, augmented_count = process_training_set(
        train_orig,
        {'images': source_paths['labeled_images'], 'annotations': source_paths['labeled_annotations']},
        train_paths
    )
    
    # Process validation set
    val_paths = {
        'images': os.path.join(preprocessed_path, "val/images"),
        'labels': os.path.join(preprocessed_path, "val/labels")
    }
    processed_val = process_dataset_split(
        val_orig,
        {'images': source_paths['labeled_images'], 'annotations': source_paths['labeled_annotations']},
        val_paths,
        "validation"
    )
    
    # Process test set
    test_paths = {
        'images': os.path.join(preprocessed_path, "test/images"),
        'labels': os.path.join(preprocessed_path, "test/labels")
    }
    processed_test = process_dataset_split(
        os.listdir(source_paths['test_images']),
        {'images': source_paths['test_images'], 'annotations': source_paths['test_annotations']},
        test_paths,
        "test"
    )
    
    # Process unlabeled set
    unlabeled_paths = {
        'images': os.path.join(preprocessed_path, "unlabeled/images")
    }
    if not os.path.exists(unlabeled_paths['images']):
        os.makedirs(unlabeled_paths['images'])
    
    processed_unlabeled = process_unlabeled_set(
        unlabeled_images,
        {'unlabeled_images': source_paths['unlabeled_images']},
        unlabeled_paths
    )
    
    # Print comprehensive dataset statistics
    print("\nFinal dataset statistics:")
    print("\nImage counts:")
    print(f"Training images (original): {processed_train}")
    print(f"Training images (augmented): {augmented_count}")
    print(f"Training images (total): {processed_train + augmented_count}")
    print(f"Validation images: {processed_val}")
    print(f"Test images: {processed_test}")
    print(f"Unlabeled images: {processed_unlabeled}")
    
    print("\nDataset sizes:")
    print(f"Training set size: {get_dir_size(os.path.join(preprocessed_path, 'train')):.2f} MB")
    print(f"Validation set size: {get_dir_size(os.path.join(preprocessed_path, 'val')):.2f} MB")
    print(f"Test set size: {get_dir_size(os.path.join(preprocessed_path, 'test')):.2f} MB")
    print(f"Unlabeled set size: {get_dir_size(os.path.join(preprocessed_path, 'unlabeled')):.2f} MB")
    print(f"Total dataset size: {get_dir_size(preprocessed_path):.2f} MB")
    
    print("\nFiles per directory:")
    for folder in folders:
        path = os.path.join(preprocessed_path, folder)
        print(f"{folder}: {len(os.listdir(path))} files")

if __name__ == "__main__":
    main()



Original dataset statistics:
Total original images: 200
Training images before augmentation: 160
Validation images: 40
Unlabeled images: 1000


Processing unlabeled images: 100%|██████████| 1000/1000 [00:08<00:00, 115.97it/s]


Final dataset statistics:

Image counts:
Training images (original): 160
Training images (augmented): 480
Training images (total): 640
Validation images: 40
Test images: 50
Unlabeled images: 1000

Dataset sizes:
Training set size: 46.42 MB
Validation set size: 2.52 MB
Test set size: 3.08 MB
Unlabeled set size: 60.48 MB
Total dataset size: 112.51 MB

Files per directory:
train/images: 640 files
train/labels: 640 files
val/images: 40 files
val/labels: 40 files
test/images: 50 files
test/labels: 50 files
unlabeled/images: 1000 files


### The data is saved in the directory named Preprocessed_img_weed ###

## Calculating F1-Score and mAP[50:95] ##

In [10]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# ----------------------------
# 1. Create data.yaml File
# ----------------------------
yaml_content = """
path: /kaggle/working/Preprocessed_img_weed
train: train
val: val
test: test

nc: 2
names: ['weed', 'crop']
"""

data_yaml_path = r'/kaggle/working/Preprocessed_img_weed/data.yaml'
if not os.path.exists(data_yaml_path):
    with open(data_yaml_path, "w") as f:
        f.write(yaml_content)
    print("data.yaml file created successfully!")
# ----------------------------
# 2. Load Models and Define Transform
# ----------------------------
YOLO_MODEL_PATH = r'/kaggle/input/yolo/pytorch/default/3/best (3).pt'   #give your YOLO model path
CLF_MODEL_PATH  = r'/kaggle/input/yolo/pytorch/default/3/best_model (4).pth'  #give your resnet model
TEST_IMAGE_DIR  = r'/kaggle/input/weedzip/test/images'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

det_model = YOLO(YOLO_MODEL_PATH)

def load_resnet50():
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 2)
    return model.to(device)

clf_model = load_resnet50()
clf_model.load_state_dict(torch.load(CLF_MODEL_PATH, map_location=device))
clf_model.eval()

clf_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# ------------------------------------------|
# 3. Process All Images and Collect Metrics |
# ----------------------------------------- |
all_true_labels = [] 
all_pred_labels = []  
all_detections = []   
all_ground_truths = []  

MIN_AREA_RATIO = 0.05

for image_filename in os.listdir(TEST_IMAGE_DIR):
    image_path = os.path.join(TEST_IMAGE_DIR, image_filename)
    image = cv2.imread(image_path)
    if image is None:
        continue
    orig_image = image.copy()
    img_h, img_w = image.shape[:2]

    results = det_model(image)
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        
        for box, score in zip(boxes, scores):
            x1, y1, x2, y2 = box.astype(int)
            box_area = (x2 - x1) * (y2 - y1)
            if box_area < MIN_AREA_RATIO * (img_w * img_h):
                continue
            if (x2-x1 < 100 or y2-y1 < 100) and ((x2-x1)/(y2-y1) < 0.5 or (x2-x1)/(y2-y1) > 2):
                continue
            
            crop = orig_image[y1:y2, x1:x2]
            if crop.size == 0:
                continue
            
            crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
            crop_pil = Image.fromarray(crop_rgb)
            input_tensor = clf_transform(crop_pil).unsqueeze(0).to(device)
            
            with torch.no_grad():
                outputs = clf_model(input_tensor)
                pred_class = int(torch.argmax(outputs, dim=1).item())
            
            all_pred_labels.append(pred_class)
            true_label = 0 if "weed" in image_filename else 1
            all_true_labels.append(true_label)
            
            all_detections.append([x1, y1, x2, y2, score])
            all_ground_truths.append(true_label)
# ----------------------------
# 4. Calculate Metrics
# ----------------------------
# F1-score for classification
f1 = f1_score(all_true_labels, all_pred_labels)
print(f"F1 Score (ResNet50): {f1:.4f}")
# mAP calculation for YOLO
metrics = det_model.val(data=data_yaml_path)


0: 640x640 1 weed, 16.2ms
Speed: 2.8ms preprocess, 16.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 background, 16.2ms
Speed: 2.6ms preprocess, 16.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 background, 16.2ms
Speed: 2.6ms preprocess, 16.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 background, 16.2ms
Speed: 2.4ms preprocess, 16.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 background, 13.6ms
Speed: 2.4ms preprocess, 13.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 background, 11.7ms
Speed: 2.4ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 weed, 11.7ms
Speed: 2.6ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 weeds, 11.7ms
Speed: 2.4ms preprocess, 11.7ms inference, 1.2ms postprocess per image at 

val: Scanning /kaggle/working/Preprocessed_img_weed/val/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.35it/s]


                   all         40         82      0.915      0.951      0.967      0.881
                  weed         14         40      0.946          1       0.98      0.924
            background         26         42      0.883      0.901      0.953      0.838


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 5.2ms preprocess, 9.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val3


## As seen F1-Score = 0.76 and mAP[50:95] = 0.881 ##
# EVALUTION_SCORE = 0.82 #